In [1]:
import torch
from torch.nn import functional as F
from src.datasets.utils import *
from src.models.bigram import BigramLM
from src.models.gpt import *
import random
import pickle

import azureml
from azureml.core import Workspace, Datastore, Experiment, Run, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute, AksCompute, ComputeTarget
from azureml.train.dnn import PyTorch

from azureml.core.webservice import Webservice, AksWebservice, AciWebservice
from azureml.core.dataset import Dataset
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.conda_dependencies import CondaDependencies 

from azureml.fsspec import AzureMachineLearningFileSystem

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.53.0


In [2]:
ws = Workspace.from_config()
print("Workspace:{}".format(ws.name))

Workspace:NanoMedGPT


In [3]:
compute_target = ComputeTarget(workspace=ws, name="GPU-Cluster")
print("Compute Target:{}".format(compute_target.name))

Compute Target:GPU-Cluster


In [4]:
ds = Datastore.get_default(ws)
print("Datastore:{}".format(ds.name))

Datastore:workspaceblobstore


In [5]:
exp = Experiment(workspace=ws, name="LocalVsCode-NanoMedGPT")

In [6]:
os.listdir(PROCESSED_DATA_PATH)

['part_0_encoded.pt',
 'part_1_encoded.pt',
 'part_2_encoded.pt',
 'part_3_encoded.pt',
 'part_4_encoded.pt',
 'part_5_encoded.pt',
 'part_6_encoded.pt']

In [19]:
""" Upload to cloud """
# from azureml.data.dataset_factory import FileDatasetFactory
# ret = FileDatasetFactory.upload_directory(
#     PROCESSED_DATA_PATH,
#     (ds, "encodings/"),
#     overwrite=True,
#     show_progress=True
# )

Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from 'c:/users/ben/git/nano_medgpt/data/processed' to 'encodings/'
Creating new dataset


In [9]:
""" Get FileDataSetFactory from cloud """
from azureml.data.dataset_factory import FileDatasetFactory
path = (ds, "encodings/")
ret = FileDatasetFactory.from_files(path)

In [13]:
type(ret)

azureml.data.file_dataset.FileDataset

In [71]:
conda_spec_file = ROOT_DIR + "\\nano_medgpt_env.yml"
pytorch_env = Environment.from_conda_specification("nano_medgpt", conda_spec_file)

In [72]:
src = ScriptRunConfig(
    source_directory= ROOT_DIR + "\\src\\",
    script = 'train.py',
    arguments=['--data-folder', ret.as_mount()],
    compute_target=compute_target,
    environment=pytorch_env
)

In [73]:
run = exp.submit(src)

In [74]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…